In [ ]:
import csv
from scipy import ndimage
from scipy import linalg
import numpy as np
import nibabel as nib
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from operator import itemgetter
import pickle

In [ ]:
#Unnecessay for the network
#for testing out if the images are actually loaded  

with open("dataimage_arr.txt", "rb") as fp:   # Unpickling
    b = pickle.load(fp)
with open("datacsv_new_arr.txt", "rb") as fp:   # Unpickling
    c = pickle.load(fp)

import matplotlib.pyplot as plt

print(type(b))
print(b[0][1][:, :, 20])
plt.imshow(b[0][1][:, :, 20])


In [ ]:
#randomly divide the patients (including their sMRI (dataimage) and informations (csv files)) into training set, 
#validation set, and testing set
import random

index = np.array(range(119))
random.seed(40)
random.shuffle(index)

train_index = index[0:84]
val_index = index[84:96]
test_index = index[96:119]


In [3]:
# reopen the sMRIs and their corresponding csv file (both in txt format) from the GPU
with open("dataimage_arr.txt", "rb") as fp:   
            dataimage = pickle.load(fp)
with open("datacsv_new_arr.txt", "rb") as fp:   
            datacsv = pickle.load(fp)
        

In [4]:
#rename csv group into numbers
mci = 0.
ad = 0.
cn = 0.
total_num = 0.
for i in range(datacsv.shape[0]):
    if datacsv[i][1] == '0':
        mci += 1.
    elif datacsv[i][1] == '1':
        ad += 1.
    elif datacsv[i][1] == '2':
        cn += 1.
total_num = mci + ad + cn
print(mci, ad, cn)

56.0 24.0 39.0


In [5]:

#resave training data (the sMRIs and their corresponding csv file (both in txt format)) to the GPU
dataimage_train = [dataimage[j] for j in train_index]
datacsv_train = [datacsv[j] for j in train_index]
with open("dataimage_train.txt", "wb") as fp:
    pickle.dump(dataimage_train, fp)
with open("datacsv_train.txt", "wb") as fp:
    pickle.dump(datacsv_train, fp)

print("finished")

finished


In [6]:
#resave validation data (the sMRIs and their corresponding csv file (both in txt format)) to the GPU
dataimage_val = [dataimage[j] for j in val_index]
datacsv_val = [datacsv[j] for j in val_index]
with open("dataimage_val.txt", "wb") as fp:
    pickle.dump(dataimage_val, fp)
with open("datacsv_val.txt", "wb") as fp:
    pickle.dump(datacsv_val, fp)
print("finished")

finished


In [7]:
#resave testing data (the sMRIs and their corresponding csv file (both in txt format)) to the GPU
dataimage_test = [dataimage[j] for j in test_index]
datacsv_test = [datacsv[j] for j in test_index]
with open("dataimage_test.txt", "wb") as fp:
    pickle.dump(dataimage_test, fp)
with open("datacsv_test.txt", "wb") as fp:
    pickle.dump(datacsv_test, fp)
print("finished")

finished


In [8]:
from torch.utils.data import Dataset, DataLoader

def ToTensor(np_array):
    tensor = torch.from_numpy(np_array)
    tensor = tensor.float()
    return tensor

class ADNI_Dataset(Dataset):
    
    def __init__(self, ptype, transform=None):
        
        if(ptype == 'train'):
            with open("dataimage_train.txt", "rb") as fp:   # Unpickling
                b_train = pickle.load(fp)
            with open("datacsv_train.txt", "rb") as fp:   # Unpickling
                c_train = pickle.load(fp)
            self.csv = c_train
            self.image = b_train
        
        if(ptype == 'val'):
            with open("dataimage_val.txt", "rb") as fp:   # Unpickling
                b_val = pickle.load(fp)
            with open("datacsv_val.txt", "rb") as fp:   # Unpickling
                c_val = pickle.load(fp)
            self.csv = c_val
            self.image = b_val
        
        if(ptype == 'test'):
            with open("dataimage_test.txt", "rb") as fp:   # Unpickling
                b_test = pickle.load(fp)
            with open("datacsv_test.txt", "rb") as fp:   # Unpickling
                c_test = pickle.load(fp)
            self.csv = c_test
            self.image = b_test        

    def __len__(self):#return int: number of mri
        return len(self.image)

    def __getitem__(self, idx):#input the dataset and desires i-th item
        img_name = self.csv[idx][0]
        image = self.image[idx][1]
        group = self.csv[idx][1]
        sample = {'image': image, 'patient_id': img_name, 'groups': group} 

        return sample

In [9]:
ADNI_Data_train = ADNI_Dataset('train')
ADNI_Data_val = ADNI_Dataset('val')
ADNI_Data_test = ADNI_Dataset('test')

dataloader_train = DataLoader(ADNI_Data_train, batch_size=1,
                        shuffle=True, num_workers=4)
dataloader_val = DataLoader(ADNI_Data_val, batch_size=1,
                        shuffle=True, num_workers=4)
dataloader_test = DataLoader(ADNI_Data_test, batch_size=1,
                        shuffle=True, num_workers=4)


In [ ]:
#design the network configuration
import torch.nn as nn
import torch.nn.functional as F

#initial: 256*256*160 * 1(feature map number)
#128*128*80 * 2
#64*64*40 * 4
#32*32*20 * 8
#16*16*10 * 16
#8*8*5 * 32
#Linear: 32 * 8 * 8 * 5 
#10000--->1024 --->256 ---> 64 --->3  
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv3d(in_channels = 1, out_channels = 2, kernel_size = 5, padding=2)
        self.conv2 = nn.Conv3d(2, 4, 5, padding=2)
        self.conv3 = nn.Conv3d(4, 8, 5, padding=2)
        self.conv4 = nn.Conv3d(8, 16, 5, padding=2)
        self.conv5 = nn.Conv3d(16, 32, 5, padding=2)
        self.pool = nn.MaxPool3d(2)
        self.fc1 = nn.Linear(32 * 8 * 8 * 5, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 3)

    def forward(self, x): #batch_size * 256*256*160*1
        x = self.pool(F.relu(self.conv1(x)))#batch_size * 128 * 128 * 80 * 2
        x = self.pool(F.relu(self.conv2(x)))#batch_size * 64 * 64 * 40 * 4
        x = self.pool(F.relu(self.conv3(x)))#batch_size * 32 * 32 * 20 * 8
        x = self.pool(F.relu(self.conv4(x)))#batch_size * 16 * 16 * 10 * 16
        x = self.pool(F.relu(self.conv5(x)))#batch_size * 8 * 8 * 5 * 32
        x = x.view(-1, 32 * 8 * 8 * 5)#batch_size * 10240(8*8*5*32)
        x = F.relu(self.fc1(x))#batch_size * 1024
        x = F.relu(self.fc2(x))#batch_size * 256
        x = F.relu(self.fc3(x))#batch_size * 64
        x = self.fc4(x)#batch_size * 3
        return x

#net = Net().cuda()

print("finished")

In [10]:
#design another network configuration

import torch.nn as nn
import torch.nn.functional as F
class Light_Net(nn.Module):
    def __init__(self):
        super(Light_Net, self).__init__()
        self.conv1 = nn.Conv3d(in_channels = 1, out_channels = 2, kernel_size = 5, padding=2)
        self.conv2 = nn.Conv3d(2, 4, 5, padding=2)
        self.conv3 = nn.Conv3d(4, 8, 5, padding=2)
        self.conv4 = nn.Conv3d(8, 8, 5, padding=2)
        self.conv5 = nn.Conv3d(8, 8, 5, padding=2)
        self.pool = nn.MaxPool3d(2)
        self.fc1 = nn.Linear(8 * 8 * 8 * 5, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 3)

    def forward(self, x): #batch_size * 256*256*160*1
        x = self.pool(F.relu(self.conv1(x)))#batch_size * 128 * 128 * 80 * 2
        x = self.pool(F.relu(self.conv2(x)))#batch_size * 64 * 64 * 40 * 4
        x = self.pool(F.relu(self.conv3(x)))#batch_size * 32 * 32 * 20 * 8
        x = self.pool(F.relu(self.conv4(x)))#batch_size * 16 * 16 * 10 * 16
        x = self.pool(F.relu(self.conv5(x)))#batch_size * 8 * 8 * 5 * 32
        x = x.view(-1, 8 * 8 * 8 * 5)#batch_size * 10240(8*8*5*32)
        x = F.relu(self.fc1(x))#batch_size * 1024
        x = F.relu(self.fc2(x))#batch_size * 256
        x = F.relu(self.fc3(x))#batch_size * 64
        x = self.fc4(x)#batch_size * 3
        return x

net = Light_Net().cuda()

In [17]:
#define the criterion and optimizer
import torch.optim as optim
import torch
Weight = torch.tensor([total_num / mci, total_num / ad, total_num / cn])
criterion = nn.CrossEntropyLoss(weight = Weight.cuda())
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

print("finished")

finished


In [19]:
import numpy as np
import torch

def ToTensor(np_array):
    tensor = torch.from_numpy(np_array)
    tensor = tensor.float()
    return tensor

epoch = 50
min_total_loss = 1000000
for i in range(epoch):
    for i_batch, sample_batched in enumerate(dataloader_train):
        print(i_batch,
              sample_batched['patient_id'])

        running_loss = 0.0
        # get the inputs
        labels_arr = np.asarray(sample_batched['groups'])
        labels_arr = labels_arr.astype(np.int)
        labels = ToTensor(labels_arr).cuda()

        images_arr = np.asarray(sample_batched['image'])
        images_arr = images_arr.astype(np.float)
        images_arr = np.expand_dims(images_arr, 1)
        images = ToTensor(images_arr).cuda()

        # zero the parameter gradients
        optimizer.zero_grad()
        
        #before actual training
        net.train()

        # forward + backward + optimize
        outputs = net(images)
        print(outputs)
        print(labels.long())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

        # print statistics
        print('[%d] loss: %.3f' %
                (i_batch + 1, loss.item()))

    print('Finished Training This Epoch')
    
    #validation for the epoch
    net.eval()
    total_val_loss = 0
    for i_batch, sample_batched in enumerate(dataloader_val):
        print(i_batch,
              sample_batched['patient_id'])

        running_loss = 0.0
        # get the inputs
        labels_arr = np.asarray(sample_batched['groups'])
        labels_arr = labels_arr.astype(np.int)
        labels = ToTensor(labels_arr).cuda()

        images_arr = np.asarray(sample_batched['image'])
        images_arr = images_arr.astype(np.float)
        images_arr = np.expand_dims(images_arr, 1)
        images = ToTensor(images_arr).cuda()
        
        outputs = net(images)
        loss = criterion(outputs, labels.long())
        print('Validation')
        print('[%d] loss: %.3f' %
                (i_batch + 1, loss.item()))
        print('\n')
        total_val_loss = total_val_loss + loss.cpu().data.numpy()
        
    print('Start a new epoch')
    #the best model is picked among all epoches
    if(total_val_loss < min_total_loss):
        min_total_loss = total_val_loss
   
    #save model
    #best_model
    torch.save(net, 'best_model_Jan12_3classes.pkl')

print("Best Model generated")

0 ['"012_S_0689"']
tensor([[ 0.1645, -0.1659,  0.0468]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[1] loss: 1.289
1 ['"037_S_0303"']
tensor([[ 0.1647, -0.1658,  0.0465]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[2] loss: 1.076
2 ['"023_S_1247"']
tensor([[ 0.1644, -0.1660,  0.0470]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[3] loss: 0.959
3 ['"002_S_1070"']
tensor([[ 0.1649, -0.1666,  0.0471]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[4] loss: 0.958
4 ['"100_S_0015"']
tensor([[ 0.1661, -0.1674,  0.0467]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[5] loss: 1.076
5 ['"023_S_0604"']
tensor([[ 0.1667, -0.1684,  0.0472]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[6] loss: 0.956
6 ['"130_S_0449"']
tensor([[ 0.1680, -0.1697,  0.0472]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cud

[63] loss: 1.322
63 ['"116_S_0649"']
tensor([[ 0.1852, -0.1966,  0.0581]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[64] loss: 0.941
64 ['"002_S_0954"']
tensor([[ 0.1863, -0.1980,  0.0585]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[65] loss: 0.940
65 ['"031_S_1209"']
tensor([[ 0.1880, -0.1996,  0.0584]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[66] loss: 1.326
66 ['"012_S_1292"']
tensor([[ 0.1890, -0.2002,  0.0580]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[67] loss: 0.938
67 ['"094_S_1293"']
tensor([[ 0.1907, -0.2010,  0.0572]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[68] loss: 0.936
68 ['"051_S_1331"']
tensor([[ 0.1930, -0.2021,  0.0560]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[69] loss: 0.934
69 ['"130_S_0956"']
tensor([[ 0.1957, -0.2034,  0.0546]], device='cuda:0', grad_fn=<ThAddmmBack

/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Light_Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


0 ['"031_S_0830"']
tensor([[ 0.2140, -0.2138,  0.0467]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[1] loss: 0.915
1 ['"005_S_0448"']
tensor([[ 0.2155, -0.2153,  0.0469]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[2] loss: 0.914
2 ['"100_S_0015"']
tensor([[ 0.2175, -0.2170,  0.0467]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[3] loss: 1.083
3 ['"126_S_0606"']
tensor([[ 0.2188, -0.2189,  0.0473]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[4] loss: 1.349
4 ['"116_S_0487"']
tensor([[ 0.2195, -0.2197,  0.0474]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[5] loss: 1.350
5 ['"067_S_1253"']
tensor([[ 0.2196, -0.2195,  0.0471]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[6] loss: 1.350
6 ['"130_S_0449"']
tensor([[ 0.2192, -0.2185,  0.0464]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cud

[56] loss: 0.918
56 ['"051_S_1331"']
tensor([[ 0.2142, -0.2426,  0.0771]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[57] loss: 0.918
57 ['"023_S_1046"']
tensor([[ 0.2150, -0.2429,  0.0766]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[58] loss: 0.918
58 ['"027_S_0417"']
tensor([[ 0.2165, -0.2435,  0.0757]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[59] loss: 0.916
59 ['"023_S_0030"']
tensor([[ 0.2185, -0.2444,  0.0745]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[60] loss: 0.914
60 ['"027_S_0404"']
tensor([[ 0.2211, -0.2455,  0.0731]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[61] loss: 1.379
61 ['"032_S_1101"']
tensor([[ 0.2229, -0.2456,  0.0713]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[62] loss: 1.379
62 ['"136_S_0196"']
tensor([[ 0.2240, -0.2448,  0.0694]], device='cuda:0', grad_fn=<ThAddmmBack

...

tensor([[ 0.3288, -0.3803,  0.1036]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[42] loss: 0.829
42 ['"023_S_0916"']
tensor([[ 0.3285, -0.3815,  0.1051]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[43] loss: 1.539
43 ['"067_S_1253"']
tensor([[ 0.3277, -0.3816,  0.1061]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[44] loss: 1.539
44 ['"007_S_1206"']
tensor([[ 0.3264, -0.3807,  0.1065]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[45] loss: 1.051
45 ['"005_S_0814"']
tensor([[ 0.3247, -0.3802,  0.1077]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[46] loss: 1.537
46 ['"116_S_0649"']
tensor([[ 0.3226, -0.3787,  0.1083]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[47] loss: 0.834
47 ['"027_S_1385"']
tensor([[ 0.3214, -0.3777,  0.1084]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[

[9] loss: 1.057
9 ['"032_S_1101"']
tensor([[ 0.3291, -0.3778,  0.1006]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[10] loss: 1.535
10 ['"027_S_0404"']
tensor([[ 0.3283, -0.3771,  0.1006]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[11] loss: 1.534
11 ['"130_S_0886"']
tensor([[ 0.3270, -0.3755,  0.1002]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[12] loss: 1.056
12 ['"100_S_0015"']
tensor([[ 0.3254, -0.3743,  0.1007]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[13] loss: 1.056
13 ['"002_S_1261"']
tensor([[ 0.3234, -0.3734,  0.1018]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[14] loss: 1.054
14 ['"005_S_0324"']
tensor([[ 0.3211, -0.3729,  0.1036]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[15] loss: 0.835
15 ['"130_S_1337"']
tensor([[ 0.3197, -0.3727,  0.1048]], device='cuda:0', grad_fn=<ThAddmmBackwa

[65] loss: 1.570
65 ['"130_S_0956"']
tensor([[ 0.3535, -0.4064,  0.1059]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[66] loss: 1.570
66 ['"027_S_0417"']
tensor([[ 0.3534, -0.4053,  0.1048]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[67] loss: 0.810
67 ['"037_S_0501"']
tensor([[ 0.3540, -0.4046,  0.1034]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[68] loss: 0.809
68 ['"094_S_1293"']
tensor([[ 0.3553, -0.4042,  0.1018]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[69] loss: 0.808
69 ['"127_S_0622"']
tensor([[ 0.3570, -0.4042,  0.1000]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[70] loss: 1.064
70 ['"136_S_0429"']
tensor([[ 0.3581, -0.4044,  0.0991]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[71] loss: 0.806
71 ['"005_S_0553"']
tensor([[ 0.3597, -0.4048,  0.0978]], device='cuda:0', grad_fn=<ThAddmmBack

tensor([[ 0.3292, -0.3694,  0.0915]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[33] loss: 0.827
33 ['"018_S_0369"']
tensor([[ 0.3307, -0.3706,  0.0912]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[34] loss: 1.065
34 ['"053_S_0507"']
tensor([[ 0.3316, -0.3719,  0.0917]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[35] loss: 0.825
35 ['"027_S_1081"']
tensor([[ 0.3330, -0.3734,  0.0918]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[36] loss: 1.530
36 ['"031_S_0830"']
tensor([[ 0.3338, -0.3737,  0.0914]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[37] loss: 0.823
37 ['"100_S_0015"']
tensor([[ 0.3351, -0.3743,  0.0907]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[38] loss: 1.066
38 ['"005_S_0553"']
tensor([[ 0.3358, -0.3751,  0.0908]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[

[11] loss: 1.094


11 ['"018_S_0450"']
Validation
[12] loss: 0.804


Start a new epoch
Best Model generated


In [22]:
import torch

#load the best_model back 
final_net = torch.load('best_model_Jan12_3classes.pkl')
final_net.eval()
correct_sum = 0.
total_sum = 0.
for i_batch, sample_batched in enumerate(dataloader_test):
    print(i_batch,
          sample_batched['patient_id'])

    running_loss = 0.0
    # get the inputs
    labels_arr = np.asarray(sample_batched['groups'])
    labels_arr = labels_arr.astype(np.int)
    labels = ToTensor(labels_arr).cuda()

    images_arr = np.asarray(sample_batched['image'])
    images_arr = images_arr.astype(np.float)
    images_arr = np.expand_dims(images_arr, 1)
    images = ToTensor(images_arr).cuda()

    outputs = final_net(images)#n * 3
    results_type = np.argmax(outputs.cpu().data.numpy(), axis=1)
    results_type = ToTensor(results_type)
    print(outputs)
    
    
    #calculate accuracy
    labels = labels.cpu()
    print("\n", results_type, " ", labels)
    correct_sum += (labels == results_type)
    total_sum += 1


print("correct_sum_final", correct_sum)
print("total_sum", total_sum)

0 ['"023_S_0058"']
tensor([[ 0.3535, -0.3670,  0.0642]], device='cuda:0', grad_fn=<ThAddmmBackward>)

 tensor([0.])   tensor([2.])
1 ['"052_S_1250"']
tensor([[ 0.3535, -0.3670,  0.0642]], device='cuda:0', grad_fn=<ThAddmmBackward>)

 tensor([0.])   tensor([2.])
2 ['"002_S_1268"']
tensor([[ 0.3535, -0.3670,  0.0642]], device='cuda:0', grad_fn=<ThAddmmBackward>)

 tensor([0.])   tensor([0.])
3 ['"100_S_1286"']
tensor([[ 0.3535, -0.3670,  0.0642]], device='cuda:0', grad_fn=<ThAddmmBackward>)

 tensor([0.])   tensor([2.])
4 ['"016_S_1117"']
tensor([[ 0.3535, -0.3670,  0.0642]], device='cuda:0', grad_fn=<ThAddmmBackward>)

 tensor([0.])   tensor([0.])
5 ['"016_S_1326"']
tensor([[ 0.3535, -0.3670,  0.0642]], device='cuda:0', grad_fn=<ThAddmmBackward>)

 tensor([0.])   tensor([0.])
6 ['"136_S_0194"']
tensor([[ 0.3535, -0.3670,  0.0642]], device='cuda:0', grad_fn=<ThAddmmBackward>)

 tensor([0.])   tensor([1.])
7 ['"027_S_0307"']
tensor([[ 0.3535, -0.3670,  0.0642]], device='cuda:0', grad_fn=<